### Spotify API

In [281]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import json
from dotenv import load_dotenv
import requests
import pandas as pd
load_dotenv()


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=os.getenv("SPOTIPY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"),
    redirect_uri="http://localhost:8889/callback",
    scope="user-top-read, user-library-read, playlist-modify-private, playlist-modify-public, playlist-read-private, playlist-read-collaborative, user-read-private, user-library-modify, ugc-image-upload"
))

token = sp.auth_manager.get_access_token()
access_token = token['access_token']
refresh_token = token['refresh_token']

headers = {
    "Authorization": f"Bearer {access_token}"
}




C:\Users\Kamil\AppData\Local\Temp\ipykernel_22380\2283351862.py:18: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token = sp.auth_manager.get_access_token()


### Get user's top tracks

In [266]:
def get_user_top_tracks(limit, time_range='long_term'):
    response = sp.current_user_top_tracks(limit=limit, time_range=time_range)
    df = pd.DataFrame(response['items'])
    df_top = df[['name', 'artists', 'popularity', 'id']]
    df['artist_name'] = df_top['artists'].apply(lambda x: x[0]['name'])
    df['artist_id'] = df_top['artists'].apply(lambda x: x[0]['id'])
    df_top_tracks = df[['name', 'artist_name','popularity', 'id', 'artist_id']]
    return df_top_tracks

# time_range: short_term, medium_term, long_term
get_user_top_tracks(20)


,name,artist_name,popularity,id,artist_id
0,homeschool,18pm,30,1BsW6UuvJasJfeH5QVM6KG,2tK4wpRrhB2TmfQ6HPhVvJ
1,Slave,UnoTheActivist,57,6gzEKpPDrMetHz0EVeo37I,0bZ9jANLkwEFVcSnHnwrpb
2,shut down entirely,meat computer,43,2iDIFn3hIbiwfsLtCggubr,7JU13ATc2v3kzIuSqNNPWN
3,I Smoked Away My Brain (I'm God x Demons Mashu...,A$AP Rocky,83,3ZaEs1O8BG581qYPHpQ8d6,13ubrt8QOOCPljQ2FL1Kca
4,For The Rush,DrownMili,29,5URibjNefwmA2fdZaXGLI9,3iotJxO373ylhqoStDSmY6
5,MANIC PIXIE DREAM BOY,SUMMER ALONE,25,58eXrciVLFZ8xfohOuFimt,7ozWZAI5CjNLOp9iFFSzek
6,Letting it all go,Yung Lean,34,1qDmrtpkcsJwH9vRS483Zf,67lytN32YpUxiSeWlKfHJ3
7,roc bottum_,Psychi,30,14eXfm84i7ngpwr1p9rogJ,3WJwLqCTu0d4pGWMCAxlnL
8,Love Sick,Gloryboyjuan,31,3i7lwPSMzHBOxBzVewAmSp,2shzFqJv7Ykg3UCLWBsEOa
9,Volt,Yung Lean,44,2OakTFuAgwO1QMQJabMKBW,67lytN32YpUxiSeWlKfHJ3


### Get user top tracks artists info

In [268]:
def get_top_tracks_artists_info(df):
    artist_ids = df['artist_id'].tolist()
    artist_info = []
    for artist_id in artist_ids:
        response = requests.get(f"https://api.spotify.com/v1/artists/{artist_id}", headers=headers)
        artist_info.append(response.json())
    df_artist_info = pd.DataFrame(artist_info)
    df_artist_info = df_artist_info[['name', 'genres', 'popularity']]
    return df_artist_info

df = get_user_top_tracks(20)
get_top_tracks_artists_info(df)

,name,genres,popularity
0,18pm,[],21
1,UnoTheActivist,"[melodic rap, plugg, rage rap]",55
2,meat computer,"[cloud rap, glitchcore, hyperpop]",44
3,A$AP Rocky,"[east coast hip hop, hip hop, rap]",86
4,DrownMili,[dark trap],29
5,SUMMER ALONE,[],44
6,Yung Lean,"[cloud rap, psychedelic hip hop, underground h...",68
7,Psychi,[],40
8,Gloryboyjuan,[],21
9,Yung Lean,"[cloud rap, psychedelic hip hop, underground h...",68


### Get user's top artists


In [269]:
def get_user_top_artists():
    response = sp.current_user_top_artists(limit=50, time_range='long_term')
    df = pd.DataFrame(response['items'])
    df = df[['name', 'genres', 'popularity']]
    return df

get_user_top_artists()

,name,genres,popularity
0,Yung Lean,"[cloud rap, psychedelic hip hop, underground h...",68
1,Radiohead,"[alternative rock, art rock, melancholia, oxfo...",85
2,$uicideboy$,"[cloud rap, dark trap, new orleans rap, underg...",87
3,Lil Peep,"[cloud rap, emo rap, lgbtq+ hip hop]",86
4,BONES,"[cloud rap, dark trap]",75
5,Playboi Carti,"[atl hip hop, plugg, pluggnb, rage rap, rap]",89
6,Bladee,"[cloud rap, drain, glitchcore, underground hip...",70
7,Eyedress,"[noise pop, pov: indie]",77
8,Mareux,[dark synthpop],66
9,68+1,[chill breakcore],46


### User's saved tracks

In [ ]:
def get_saved_tracks(limit):
    response = sp.current_user_saved_tracks(limit=limit)
    df = pd.DataFrame(response['items'])
    df = df[['track']]
    df['name'] = df['track'].apply(lambda x: x['name'])
    df['artist_name'] = df['track'].apply(lambda x: x['artists'][0]['name'])
    df['artist_id'] = df['track'].apply(lambda x: x['artists'][0]['id'])
    df['id'] = df['track'].apply(lambda x: x['id'])
    df = df[['name', 'artist_name', 'artist_id', 'id']]
    return df

def get_playlist_tracks(playlist_id, limit):
    response = sp.playlist_tracks(playlist_id, limit=limit, offset=20)
    df = pd.DataFrame(response['items'])
    df = df[['track']]
    df['name'] = df['track'].apply(lambda x: x['name'])
    df['artist_name'] = df['track'].apply(lambda x: x['artists'][0]['name'])
    df['artist_id'] = df['track'].apply(lambda x: x['artists'][0]['id'])
    df['id'] = df['track'].apply(lambda x: x['id'])
    df = df[['name', 'artist_name', 'artist_id', 'id']]
    return df

def get_user_playlists():
    response = sp.current_user_playlists()
    df = pd.DataFrame(response['items'])
    df = df[['name', 'id']]
    return df




### LAST.FM test

In [ ]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()
import hashlib

API_KEY = os.getenv("LASTFM_API_KEY")
API_SECRET = os.getenv("LASTFM_API_SECRET")

method = 'auth.getToken'
params = {
    'method': method,
    "api_key": API_KEY,
    "format": "json"
}
sorted_params = sorted(params.items())
concatenated_params = ''.join(f'{k}{v}' for k, v in sorted_params) + API_SECRET
api_sig = hashlib.md5(concatenated_params.encode('utf-8')).hexdigest()

# Wysłanie zapytania
params['api_sig'] = api_sig

response = requests.get('http://ws.audioscrobbler.com/2.0/', params=params)

# Odczytanie odpowiedzi
data = response.json()
if 'token' in data:
    print(f"Token wstępny: {data['token']}")
else:
    print("Błąd w uzyskaniu tokenu wstępnego:", data)

authorization_url = f'http://www.last.fm/api/auth?api_key={API_KEY}&token={data["token"]}'
print(f'Aby autoryzować aplikację, przejdź do tego URL: {authorization_url}')


In [ ]:
import requests
import pandas as pd
def get_similar_tracks(artist_name, track_name, limit):
    api_url = "https://ws.audioscrobbler.com/2.0/"
    params = {
        'method': 'track.getsimilar',
        'artist': artist_name,
        'track': track_name,
        'api_key': API_KEY,
        'limit': limit,
        'format': 'json',
    }

    response = requests.get(api_url, params=params)
    df = pd.DataFrame(response.json()['similartracks']['track'])
    df = df[['name', 'artist', 'url']]
    df['artist_name'] = df['artist'].apply(lambda x: x['name'])
    df = df[['name', 'artist_name', 'url']]
    return df





### Create a playlist and add a track

In [ ]:

def get_playlist_id(name):
    playlists = sp.current_user_playlists()
    for playlist in playlists['items']:
        if playlist['name'] == name:
            return playlist['id']
    return None


def create_lastfm_playlist(name):
    user_id = requests.get('https://api.spotify.com/v1/me', headers=headers).json()['id']
    sp.user_playlist_create(user_id, name, public=False)
    if get_playlist_id(name):
        return f"Playlist {name} created"
    else:
        return f"Playlist {name} not created"


def get_track_id(track_name, artist_name):
    track_id = sp.search(q=f"track:{track_name} artist:{artist_name}", type='track')
    df = pd.DataFrame(track_id['tracks']['items'])
    if not df.empty:
        df_result = df[['id', 'external_urls']].copy()
        df_result.loc[:, 'url'] = df_result['external_urls'].apply(lambda x: x['spotify'] if 'spotify' in x else None)
        return df_result[['id', 'url']]
    else:
        return pd.DataFrame(columns=['id', 'url'])

def add_track_to_playlist(playlist_id, track_id):
    return sp.playlist_add_items(playlist_id, [track_id])


### Get recommendations

In [ ]:
def info_to_lastfm(data, limit):
    for name,artist_name in data[['name', 'artist_name']].values:
        return get_similar_tracks(artist_name, name, limit=limit)


def to_playlist(data, to_playlist):
    playlist_id = get_playlist_id(to_playlist)
    for name, artist_name in data[['name', 'artist_name']].values:
        df = get_track_id(name, artist_name)
        if not df.empty:
            track_id = df['id'].values[0]
            add_track_to_playlist(playlist_id, track_id)
        else:
            print(f"No track found for {name} by {artist_name}")



### Create a playlist with recommendations


In [ ]:

def recommand_playlist_from_playlist(from_playlist_name, to_playlist_name=None, limit_recommand=20, limit_playlist=40):
    from_playlist_id = get_playlist_id(from_playlist_name)
    to_playlist_id = get_playlist_id(to_playlist_name)
    if from_playlist_id is None:
        print(f"Playlist '{from_playlist_name}' not found.")
        return None

    if to_playlist_id is not None:
        playlist_tracks = get_playlist_tracks(from_playlist_id, limit=limit_playlist)
        playlist_add = info_to_lastfm(playlist_tracks, limit_recommand)
        to_playlist(playlist_add, to_playlist_name)
    else:
        create_lastfm_playlist(to_playlist_name)
        playlist_tracks = get_playlist_tracks(from_playlist_id, limit=limit_playlist)
        playlist_add = info_to_lastfm(playlist_tracks, limit_recommand)
        to_playlist(playlist_add, to_playlist_name)

    return f"Playlist '{to_playlist_name}' created with {limit_recommand} tracks"

def recommand_playlist_from_saved_tracks(to_playlist_name, limit_recommand=20, limit_playlist=40):
    to_playlist_id = get_playlist_id(to_playlist_name)
    if to_playlist_id is not None:
        saved_tracks = get_saved_tracks(limit=limit_playlist)
        playlist_add = info_to_lastfm(saved_tracks, limit_recommand)
        to_playlist(playlist_add, to_playlist_name)
    else:
        create_lastfm_playlist(to_playlist_name)
        saved_tracks = get_saved_tracks(limit=limit_playlist)
        playlist_add = info_to_lastfm(saved_tracks, limit_recommand)
        to_playlist(playlist_add, to_playlist_name)

    return f"Playlist '{to_playlist_name}' created with {limit_recommand} tracks"


### Edit playlist info


In [292]:
import io
from PIL import Image
import requests
import base64


def edit_cover_image(playlist_id, image_path):
    with Image.open(image_path) as img:
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format=img.format)
        img_size = img_byte_arr.tell()

        if img_size > 256 * 1024:
            resize_factor = (256 * 1024) / img_size
            new_width = int(img.width * resize_factor ** 0.5)
            new_height = int(img.height * resize_factor ** 0.5)
            img = img.resize((new_width, new_height), Image.ANTIALIAS)

            img_byte_arr = io.BytesIO()
            img.save(img_byte_arr, format='JPEG')

        img_byte_arr.seek(0)
        img_base64 = base64.b64encode(img_byte_arr.read()).decode('utf-8')

        sp.playlist_upload_cover_image(playlist_id, img_base64)
        return "Cover image uploaded"

#edit_cover_image("4kcdTZrJx3ziJIkVATKzQo", "./data/uwu.png")

def edit_description(playlist_id, description):
    sp.playlist_change_details(playlist_id, description=description)
    return "Description updated"

edit_description("4kcdTZrJx3ziJIkVATKzQo", "This is a playlist created by a bot")

def edit_name(playlist_id, name):
    sp.playlist_change_details(playlist_id, name=name)
    return "Name updated"

def edit_collaborative(playlist_id, collaborative):
    sp.playlist_change_details(playlist_id, collaborative=collaborative)
    return "Collaborative updated"

def edit_public(playlist_id, public):
    sp.playlist_change_details(playlist_id, public=public)
    return "Public updated"

def edit_playlist_info(playlist_id, name=None, description=None, collaborative=None, public=None, edit_image=None):
    if name:
        edit_name(playlist_id, name)
    if description:
        edit_description(playlist_id, description)
    if collaborative:
        edit_collaborative(playlist_id, collaborative)
    if public:
        edit_public(playlist_id, public)
    if edit_image:
        edit_cover_image(playlist_id, edit_image)

    return "Playlist info updated"

edit_playlist_info("4kcdTZrJx3ziJIkVATKzQo", name="Lightheaded", description="This is a playlist created by a bot",
edit_image="./data/uwu.png", collaborative=False, public=False)

'Playlist info updated'

# Usage


#### Fetch Similar Tracks from Last.fm

In [273]:
similar_tracks = get_similar_tracks(artist_name='Rosé', track_name='APT.', limit=5)
similar_tracks

,name,artist_name,url
0,Toxic Till The End,Rosé,https://www.last.fm/music/Ros%C3%A9/_/Toxic+Ti...
1,TOO BAD FOR US,Rosé,https://www.last.fm/music/Ros%C3%A9/_/TOO+BAD+...
2,tAsTe,Sabrina Carpenter,https://www.last.fm/music/Sabrina+Carpenter/_/...
3,Imaginary Friend,Itzy,https://www.last.fm/music/Itzy/_/Imaginary+Friend
4,Juno,Sabrina Carpenter,https://www.last.fm/music/Sabrina+Carpenter/_/...


#### Get user's playlists

In [274]:
get_user_playlists()

,name,id
0,Week one,3HT00kaWys2eB6A1N8WkVB
1,test,2DhWAYDxwVhtFBFtAVvRBO
2,Test playlist,4O9IMtvGyKTW8zxNyW2aWh
3,🫨,2LkFoLe6vLTUMTKhg1MHkF
4,All time best,5W6EWwcVwaoq5I1cZ83ak4
5,Japanese Radio Songs,2AYRExp3Zl7ZaBRhj8zwUa
6,Kolanoskopia w Paryżu,09EY9hVmpnpIui9dV8Id7U
7,Lightheaded,4kcdTZrJx3ziJIkVATKzQo
8,BreakMe Part 2,55CZWCNS2BqC7JjAISDE3Z
9,My 2023 Playlist in a Bottle,47uiQOXVneKwyqm5SRUIpT


#### Get user's playlist tracks

In [276]:
get_playlist_tracks(playlist_id="3HT00kaWys2eB6A1N8WkVB", limit=10)

,name,artist_name,artist_id,id
0,Don't Go Outside,POORSTACY,7vSY9HEreOqb1Llar3UC38,0g43TI4HIf2BCY9kPaKt1Y
1,It Could Be Worse,POORSTACY,7vSY9HEreOqb1Llar3UC38,2DRDeWqxF83hjgIaxqzQFD
2,Urself (feat. POORSTACY),iann dior,6ASri4ePR7RlsvIQgWPJpS,7DM1OlB2Cyv7Km9itQuXzy
3,Last Night,Tom The Mail Man,1ueFyDvrq8tCjAd6x8AVxD,3n6d5aO3kza3b5tWxe0tg5
4,Lost,RONEN,3wDeS1sr05fxaOeiJJrOnL,0ytmxOjS0L12GlEdGQJo37
5,Hidden Souls,RONEN,3wDeS1sr05fxaOeiJJrOnL,2b3VM9hDGCEvObJ7AAMLyZ
6,•NOTDEAD!•,Tom The Mail Man,1ueFyDvrq8tCjAd6x8AVxD,6fRVwblLIfC5DrVH8swfFs
7,Swan Dive,convolk,3ddT1Q3KQAm1G7UcIfz5KJ,1IzGdhQ17U0r7ufI4qBv7d
8,nothinglastsforever,TOKYO’S REVENGE,5TbLOwv8BNnik3f03NZJlt,1tSbjFhk9MJ8x7WfpPNByI
9,Darkside (feat. Travis Barker),iann dior,6ASri4ePR7RlsvIQgWPJpS,614MG3boQl28AijRF1BJbS


#### Get user's saved tracks


In [277]:
get_saved_tracks(limit=10)

,name,artist_name,artist_id,id
0,You're Not The Only One I Know,The Sundays,4nlqDmbzFzbNITaqjJv7D7,3nbFVNHVYCMCdD9vBo3udv
1,Kool-Aid,Bring Me The Horizon,1Ffb6ejR6Fe5IamqA5oRUF,0hpWmAB3L0OJ3VBeMkOQUu
2,Can You Feel My Heart,Bring Me The Horizon,1Ffb6ejR6Fe5IamqA5oRUF,0WSa1sucoNRcEeULlZVQXj
3,If You Can't Hang,Sleeping With Sirens,3N8Hy6xQnQv1F1XCiyGQqA,2Tc9VznHtQUmfOgE3L1RdN
4,Untitled #3,Girlfriends,02eZaSqr4w3FTDeabE8KjG,5tRN9XGTvDm3oIPZWsoEYV
5,New World,Aloboi,4Lfqvnzd6MN7hmHJt6LfcY,7t2m5GPuIZsH9hqsHNnP4y
6,Dies Irae - Just Raw,Aloboi,4Lfqvnzd6MN7hmHJt6LfcY,6GJWln4OB1sN5x5AEh3v50
7,Distant Light,Prado,4uwqPAW8ic3uAi8bUdAZu9,0hFuuQCY1QAwuetAg0gAfP
8,Various Types Of Ads,Rory in early 20s,2E701AAAlg7LthbISEZv0N,79VIJ0S4PAKXuvKpmfsBRD
9,Da Da Is Tape To And U U U,Rory in early 20s,2E701AAAlg7LthbISEZv0N,122HlfYjPsedvPdbAJgswz


#### Create a playlist with recommendations

In [278]:
recommand_playlist_from_playlist(
    from_playlist_name="Kuba i Kamcio",
    to_playlist_name="Week two",
    limit_recommand=50,
    limit_playlist=50
)

No track found for Lovin' Times (Mythos'n DJ Cosmo Edit Version) by Web
No track found for Larks' Tongues In Aspic, Part Four by King Crimson
No track found for Travelling Lady by Manfred Mann's Earth Band
No track found for True to the Click by Brand X


"Playlist 'Week two' created with 50 tracks"

#### Add tracks to existing playlist

In [ ]:
data = get_user_top_tracks(limit=10)
to_playlist(data, to_playlist="My Recommendations")